### Tarea 9 
#### Miguel Castañeda Santiago  175840

## Ejercicio/Tarea

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?

#### Para realizar la tarea se genera un cluster con tres nodos uno funcionando como scheduler y worker

![](cluster.png)

Los datos se tomaron de  s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

#### Secuencial

In [1]:
import pandas as pd
from sklearn.linear_model import *
from sklearn.pipeline import *
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
%time tripdata_pd_df = pd.read_csv("/tmp/yellow_tripdata_2015-01.csv")

CPU times: user 31 s, sys: 2.3 s, total: 33.3 s
Wall time: 33.5 s


In [3]:
%time len(tripdata_pd_df)

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 16 µs


12748986

In [4]:
tripdata_pd_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.00,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.00,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.00,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.00,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.00,0.3,16.30
5,1,2015-01-10 20:33:39,2015-01-10 20:53:52,1,9.00,-73.874374,40.774048,1,N,-73.986977,40.758194,1,27.0,0.5,0.5,6.70,5.33,0.3,40.33
6,1,2015-01-10 20:33:39,2015-01-10 20:58:31,1,2.20,-73.983276,40.726009,1,N,-73.992470,40.749634,2,14.0,0.5,0.5,0.00,0.00,0.3,15.30
7,1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.80,-74.002663,40.734142,1,N,-73.995010,40.726326,1,7.0,0.5,0.5,1.66,0.00,0.3,9.96
8,1,2015-01-10 20:33:39,2015-01-10 21:11:35,3,18.20,-73.783043,40.644356,2,N,-73.987595,40.759357,2,52.0,0.0,0.5,0.00,5.33,0.3,58.13
9,1,2015-01-10 20:33:40,2015-01-10 20:40:44,2,0.90,-73.985588,40.767948,1,N,-73.985916,40.759365,1,6.5,0.5,0.5,1.55,0.00,0.3,9.35


#### Preparar los datos

In [19]:
tripdata_pd_df.tpep_pickup_datetime = tripdata_pd_df.tpep_pickup_datetime.astype('M8[us]')
tripdata_pd_df.tpep_dropoff_datetime = tripdata_pd_df.tpep_dropoff_datetime.astype('M8[us]')

In [20]:
## Proporción de la propina
tripdata_pd_df["tip_prop"] = tripdata_pd_df.tip_amount / tripdata_pd_df.total_amount
## Duración del viaje
tripdata_pd_df["trip_duration"] = tripdata_pd_df.tpep_dropoff_datetime - tripdata_pd_df.tpep_pickup_datetime
## Duración del viaje en segundos
tripdata_pd_df["trip_duration_sec"] = tripdata_pd_df.trip_duration.map(lambda d: d.total_seconds())

In [8]:
tripdata_pd_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,tip_prop,trip_duration,trip_duration_sec
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,...,12.0,1.0,0.5,3.25,0.0,0.3,17.05,0.190616,00:18:03,1083.0
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,...,14.5,0.5,0.5,2.00,0.0,0.3,17.80,0.112360,00:19:50,1190.0
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,...,9.5,0.5,0.5,0.00,0.0,0.3,10.80,0.000000,00:10:03,603.0
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,...,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.000000,00:01:52,112.0
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,...,15.0,0.5,0.5,0.00,0.0,0.3,16.30,0.000000,00:19:19,1159.0


Considerando las variables:

"VendorID"
"passenger_count"
"trip_distance"
"RateCodeID"
"fare_amount"
"extra"
"tolls_amount"
"total_amount"
"trip_duration_sec"
"tip_amount"


In [9]:
tripdata_pd_df[["VendorID", "passenger_count", "trip_distance", "RateCodeID", "fare_amount", "extra", "tolls_amount", "total_amount", "trip_duration_sec","tip_amount"]].head(10)

,VendorID,passenger_count,trip_distance,RateCodeID,fare_amount,extra,tolls_amount,total_amount,trip_duration_sec,tip_amount
0,2,1,1.59,1,12.0,1.0,0.00,17.05,1083.0,3.25
1,1,1,3.30,1,14.5,0.5,0.00,17.80,1190.0,2.00
2,1,1,1.80,1,9.5,0.5,0.00,10.80,603.0,0.00
3,1,1,0.50,1,3.5,0.5,0.00,4.80,112.0,0.00
4,1,1,3.00,1,15.0,0.5,0.00,16.30,1159.0,0.00
5,1,1,9.00,1,27.0,0.5,5.33,40.33,1213.0,6.70
6,1,1,2.20,1,14.0,0.5,0.00,15.30,1492.0,0.00
7,1,3,0.80,1,7.0,0.5,0.00,9.96,521.0,1.66
8,1,3,18.20,2,52.0,0.0,5.33,58.13,2276.0,0.00
9,1,2,0.90,1,6.5,0.5,0.00,9.35,424.0,1.55


In [10]:
X_train, X_test, y_train, y_test = train_test_split(tripdata_pd_df[["VendorID", "passenger_count", "trip_distance", "RateCodeID", "fare_amount", "extra", "tolls_amount", "total_amount", "trip_duration_sec"]],
                                                    tripdata_pd_df.tip_amount,
                                                    test_size=0.3, random_state=175840)

In [11]:
print("Entrenamiento: " + str(len(X_train)))
print("Prueba: " + str(len(X_test)))

Entrenamiento: 8924290
Prueba: 3824696


In [13]:
def magic_loop(X_test,y_test,X_train, y_train):
    grid_params = { "lasso__eps": [1e-3, 1e-4],"lasso__n_alphas": [50, 100]}
    lasso = LassoCV(max_iter=1000, n_alphas=100, n_jobs=10, random_state=175840)
    pipeline = Pipeline([("lasso", lasso)])
    grid = GridSearchCV(estimator=pipeline, param_grid=grid_params, n_jobs=10, scoring="neg_mean_squared_error")
    gridm = grid.fit(X_train, y_train)
    print("Score: %f con métricas '%s'\nLos mejores parámetros son: %s"%(gridm.score(X_test, y_test), grid.scoring,gridm.best_params_))

Al ejecutar el conjunto de datos iniciales marcó un error de memoria la busqueda de hiperparámetros, se selecciona una muestra del 30%

In [14]:
tripdata_pd_df_s = tripdata_pd_df.sample(frac=0.3,replace=False)

In [15]:
len(tripdata_pd_df_s)

3824696

In [16]:
X_train, X_test, y_train, y_test = train_test_split(tripdata_pd_df_s[["VendorID", "passenger_count", "trip_distance", "RateCodeID", "fare_amount", "extra", "tolls_amount", "total_amount", "trip_duration_sec"]],
                                                    tripdata_pd_df_s.tip_amount,
                                                    test_size=0.3, random_state=175840)

In [17]:
print("Entrenamiento: " + str(len(X_train)))
print("Prueba: " + str(len(X_test)))

Entrenamiento: 2677287
Prueba: 1147409


In [18]:
%time magic_loop(X_train,y_train,X_train, y_train)

Score: -0.041211 con métricas 'neg_mean_squared_error'
Los mejores parámetros son: {'lasso__eps': 0.0001, 'lasso__n_alphas': 50}
CPU times: user 47 s, sys: 3.9 s, total: 50.9 s
Wall time: 2min 43s


### Paralelo

Usando el mismo conjunto de datos y usando el decorador delayed

In [1]:
from dask import delayed
from dask.distributed import Client

client = Client("tcp://192.168.0.107:8786")

@delayed
def delayed_magic_loop(X_test,y_test,X_train, y_train):
    grid_params = { "lasso__eps": [1e-3, 1e-4],"lasso__n_alphas": [50, 100]}
    lasso = LassoCV(max_iter=1000, n_alphas=100, n_jobs=10, random_state=175840)
    pipeline = Pipeline([("lasso", lasso)])
    grid = GridSearchCV(estimator=pipeline, param_grid=grid_params, n_jobs=10, scoring="neg_mean_squared_error")
    gridm = grid.fit(X_train, y_train)
    print("Score: %f con métricas '%s'\nLos mejores parámetros son: %s"%(gridm.score(X_test, y_test), grid.scoring,gridm.best_params_))

In [23]:
%time delayed_magic_loop(X_test,y_test,X_train, y_train).compute()

CPU times: user 731 ms, sys: 709 ms, total: 1.44 s
Wall time: 4min 35s


### Usando  Dask-ML

Preparamos los datos ahora usando un dataframe de dask

In [2]:
from dask import dataframe

In [3]:
tripdata_df = dataframe.read_csv("/tmp/yellow_tripdata_2015-01.csv")

In [4]:
tripdata_df.tpep_pickup_datetime = tripdata_df.tpep_pickup_datetime.astype('M8[us]')
tripdata_df.tpep_dropoff_datetime = tripdata_df.tpep_dropoff_datetime.astype('M8[us]')
tripdata_df["tip_prop"] = tripdata_df.tip_amount / tripdata_df.total_amount
tripdata_df["trip_duration"] = tripdata_df.tpep_dropoff_datetime - tripdata_df.tpep_pickup_datetime
tripdata_df["trip_duration_sec"] = tripdata_df.trip_duration.map(lambda d: d.total_seconds())

In [5]:
tripdata_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,tip_prop,trip_duration,trip_duration_sec
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,...,12.0,1.0,0.5,3.25,0.00,0.3,17.05,0.190616,00:18:03,1083.0
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,...,14.5,0.5,0.5,2.00,0.00,0.3,17.80,0.112360,00:19:50,1190.0
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,...,9.5,0.5,0.5,0.00,0.00,0.3,10.80,0.000000,00:10:03,603.0
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,...,3.5,0.5,0.5,0.00,0.00,0.3,4.80,0.000000,00:01:52,112.0
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,...,15.0,0.5,0.5,0.00,0.00,0.3,16.30,0.000000,00:19:19,1159.0
5,1,2015-01-10 20:33:39,2015-01-10 20:53:52,1,9.00,-73.874374,40.774048,1,N,-73.986977,...,27.0,0.5,0.5,6.70,5.33,0.3,40.33,0.166129,00:20:13,1213.0
6,1,2015-01-10 20:33:39,2015-01-10 20:58:31,1,2.20,-73.983276,40.726009,1,N,-73.992470,...,14.0,0.5,0.5,0.00,0.00,0.3,15.30,0.000000,00:24:52,1492.0
7,1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.80,-74.002663,40.734142,1,N,-73.995010,...,7.0,0.5,0.5,1.66,0.00,0.3,9.96,0.166667,00:08:41,521.0
8,1,2015-01-10 20:33:39,2015-01-10 21:11:35,3,18.20,-73.783043,40.644356,2,N,-73.987595,...,52.0,0.0,0.5,0.00,5.33,0.3,58.13,0.000000,00:37:56,2276.0
9,1,2015-01-10 20:33:40,2015-01-10 20:40:44,2,0.90,-73.985588,40.767948,1,N,-73.985916,...,6.5,0.5,0.5,1.55,0.00,0.3,9.35,0.165775,00:07:04,424.0


In [13]:
tripdata_df[["VendorID", "passenger_count", "trip_distance", "RateCodeID", "fare_amount", "extra", "tolls_amount", "total_amount", "trip_duration_sec","tip_amount"]].head(10)

,VendorID,passenger_count,trip_distance,RateCodeID,fare_amount,extra,tolls_amount,total_amount,trip_duration_sec,tip_amount
0,2,1,1.59,1,12.0,1.0,0.00,17.05,1083.0,3.25
1,1,1,3.30,1,14.5,0.5,0.00,17.80,1190.0,2.00
2,1,1,1.80,1,9.5,0.5,0.00,10.80,603.0,0.00
3,1,1,0.50,1,3.5,0.5,0.00,4.80,112.0,0.00
4,1,1,3.00,1,15.0,0.5,0.00,16.30,1159.0,0.00
5,1,1,9.00,1,27.0,0.5,5.33,40.33,1213.0,6.70
6,1,1,2.20,1,14.0,0.5,0.00,15.30,1492.0,0.00
7,1,3,0.80,1,7.0,0.5,0.00,9.96,521.0,1.66
8,1,3,18.20,2,52.0,0.0,5.33,58.13,2276.0,0.00
9,1,2,0.90,1,6.5,0.5,0.00,9.35,424.0,1.55


In [15]:
import pandas as pd
%time tripdata_pd_df = pd.read_csv("/tmp/yellow_tripdata_2015-01.csv")


CPU times: user 41.9 s, sys: 2.6 s, total: 44.5 s
Wall time: 45.4 s


In [21]:
tripdata_pd_df_s = tripdata_pd_df.sample(frac=0.3,replace=False)
X_train, X_test, y_train, y_test = train_test_split(tripdata_pd_df_s[["VendorID", "passenger_count", "trip_distance", "RateCodeID", "fare_amount", "extra", "tolls_amount", "total_amount", "trip_duration_sec"]],
                                                    tripdata_pd_df_s.tip_amount,
                                                    test_size=0.3, random_state=175840)

In [22]:
print("Entrenamiento: " + str(len(X_train)))
print("Prueba: " + str(len(X_test)))

Entrenamiento: 2677287
Prueba: 1147409


In [33]:
from dask_searchcv import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import *
from sklearn.pipeline import *

In [36]:
def daskml_magic_loop(X_test,y_test,X_train, y_train):
    grid_params = { "lasso__eps": [1e-3, 1e-4],"lasso__n_alphas": [50, 100]}
    lasso = LassoCV(max_iter=1000, n_alphas=100, n_jobs=10, random_state=175840)
    pipeline = Pipeline([("lasso", lasso)])
    grid = GridSearchCV(estimator=pipeline, param_grid=grid_params, n_jobs=10, scoring="neg_mean_squared_error")
    gridm = grid.fit(X_train, y_train)
    print("Score: %f con métricas '%s'\nLos mejores parámetros son: %s"%(gridm.score(X_test, y_test), grid.scoring,gridm.best_params_))

In [37]:
%time daskml_magic_loop(X_test,y_test,X_train, y_train)

Score: -0.085858 con métricas 'neg_mean_squared_error'
Los mejores parámetros son: {'lasso__eps': 0.0001, 'lasso__n_alphas': 50}
CPU times: user 934 ms, sys: 428 ms, total: 1.36 s
Wall time: 2min 33s


#### Conclusión


Utilizar dask_ml mejora el tiempo de respuesta, para el caso de paralelo y usando el decorador "delayed" obtuve practicamente el mismo tiempo de respuesta. El ejercicio se realizó utilizando tres nodos y los datos se leen en cada nodo desde la ruta /tmp, no alcance a hacer la prueba usando hdfs, adicionalmente en el ejercicio el uso de la memoria no alcanzo nunca el 100% del cluster así como el uso del procesador de dos nodos se mantuvo por debajo del 20% por lo que seguramente me faltó trabajar en revisar los parámetros del cluster para obtener un mejor aprovechamiento.